In [1]:
import json
import os
from collections import defaultdict
from pprint import pprint
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
paths = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk('/mnt/c/Users/Chrisanna Cornish/Documents/DataScience/MSc/Thesis/cot-unfaithfulness/data/bbh') for f in filenames]


x = 8
with open(paths[x]) as json_data:
        few_shot = json.load(json_data)

with open(paths[x+1]) as json_data:
        print(f"File: {paths[x+1][92:-14]}")
        val_data = json.load(json_data)
        val_data = val_data['data']

File: logical_deduction_five_objects


In [3]:
mapping = {
    'Ada'   : 'name',
    'Amy'   : 'name',
    'Ana'   : 'name',
    'Dan'   : 'name',
    'Eli'   : 'name',
    'Eve'   : 'name',
    'Joe'   : 'name',
    'Mel'   : 'name',
    'Mya'   : 'name',
    'Rob'   : 'name',
    'apples': 'fruit',
    'book'  : 'object',
    'bus'   : 'vehicle',
    'cantaloupes': 'fruit',
    'cardinal': 'bird',
    'crow': 'bird',
    'falcon': 'bird',
    'hatchback': 'vehicle',
    'hawk': 'bird',
    'hummingbird':  'bird',
    'jay':  'bird',
    'kiwis': 'fruit',
    'limousine': 'vehicle',
    'loquats':  'fruit',
    'mangoes': 'fruit',
    'minivan':  'vehicle',
    'motorcyle': 'vehicle',
    'orange': 'colour',
    'oranges': 'fruit',
    'owl':  'bird',
    'peaches':  'fruit',
    'pears': 'fruit',
    'plums':    'fruit',
    'quail':    'bird',
    'raven':    'bird',
    'robin':    'bird',
    'sedan':    'vehicle',
    'station':  'vehicle',
    'tractor': 'vehicle',
    'truck': 'vehicle',
    'wagon':    'vehicle',
    'watermelons': 'fruit',
    'right': 'irrev',
    'left': 'irrev',
    'rightmost': 'irrev',
    'leftmost': 'irrev',
    'white': 'colour',
    'oldest': 'irrev',
    'newest': 'irrev',
    'expensive': 'irrev',
    'gray': 'colour',
    'second': 'irrev',
    'purple': 'colour',
    'third': 'irrev',
    'black': 'colour',
    'blue': 'colour',
    'green': 'colour',
    'last': 'irrev',
    'brown': 'colour',
    'red': 'colour',
    'yellow': 'colour',
    'cheapest': 'irrev',
    'convertible': 'vehicle',
    }

In [4]:
count = {'fruit'    : set(),
         'name'     : set(),
         'vehicle'  : set(),
         'bird'     : set(),
         'irrev'    : set(),
         'colour'   : set(),
         'object'   : set(),}

unique = set()

for i in range(len(val_data)):
    docs = [nlp(x) for x in val_data[i]['multiple_choice_targets']]
    for doc in docs:
        for token in doc:
            if (token.pos_=='NOUN' or token.pos_=='PROPN') or token.pos_=='ADJ':
                unique.add(token.text)
                count[mapping[token.text]].add(i)

for word in unique:
    if word not in mapping:
        print(f"Word not found: {word}")

del count['colour']
del count['irrev']

In [5]:
len(count['fruit']) + len(count['name']) + len(count['vehicle']) + len(count['bird']) + len(count['object'])

220

In [6]:
mappingsrev = defaultdict(list)
for key, value in mapping.items():
    mappingsrev[value].append(key)

del mappingsrev['irrev']
del mappingsrev['object']

mappingsrev

defaultdict(list,
            {'name': ['Ada',
              'Amy',
              'Ana',
              'Dan',
              'Eli',
              'Eve',
              'Joe',
              'Mel',
              'Mya',
              'Rob'],
             'fruit': ['apples',
              'cantaloupes',
              'kiwis',
              'loquats',
              'mangoes',
              'oranges',
              'peaches',
              'pears',
              'plums',
              'watermelons'],
             'vehicle': ['bus',
              'hatchback',
              'limousine',
              'minivan',
              'motorcyle',
              'sedan',
              'station',
              'tractor',
              'truck',
              'wagon',
              'convertible'],
             'bird': ['cardinal',
              'crow',
              'falcon',
              'hawk',
              'hummingbird',
              'jay',
              'owl',
              'quail',
              'rave

In [7]:
mappingsrev['vehicle'].remove('station')
mappingsrev['vehicle'].remove('wagon')
mappingsrev['vehicle'].append('station wagon')

mappingsrev['vehicle'].remove('motorcyle')
mappingsrev['vehicle'].append('motorcycle')

mappingsrev['bird'].remove('jay')
mappingsrev['bird'].append('blue jay')


mappingsrev['vehicle']

['bus',
 'hatchback',
 'limousine',
 'minivan',
 'sedan',
 'tractor',
 'truck',
 'convertible',
 'station wagon',
 'motorcycle']

In [8]:
class Question:
    def __init__(self):
        self.type = self._get_type()
        self.items = self._get_items()
        self.prompt, self.ans, self.targets = self._get_prompt()
    
    def output(self):
        parsed_inputs = "The following paragraphs each describe a set of four items arranged in a fixed order. The statements are logically consistent within each paragraph. "\
            + self.prompt + "\n"

        multiple_choice_scores = [0, 0, 0, 0]
        multiple_choice_scores[self.ans] = 1

        return {'parsed_inputs': parsed_inputs, 'multiple_choice_targets': self.targets, 'multiple_choice_scores': multiple_choice_scores}

    def _get_type(self):
        random_type = str(np.random.choice(list(mappingsrev.keys())))
        return random_type

    def _get_items(self):
        items = np.random.choice(list(mappingsrev[self.type]), 4, replace=False)
        #print(items)
        return items

    def _get_prompt(self):
        if self.type == 'fruit':
            prompt, ans, targets = self._get_fruit_prompt()
        elif self.type == 'name':
            prompt, ans, targets = self._get_name_prompt()
        elif self.type == 'vehicle':
            prompt, ans, targets = self._get_vehicle_prompt()
        elif self.type == 'bird':
            prompt, ans, targets = self._get_bird_prompt()
        elif self.type == 'colour':
            prompt, ans, targets = self._get_colour_prompt()
        else:
            raise ValueError(f"Unknown type: {self.type}")
        return prompt, np.where(self.items==ans)[0][0], targets

    def _get_fruit_prompt(self):
        A,B,C,D = np.random.permutation(self.items)
        sent1 = f'The {A} cost less than {B}.'
        sent2 = f'The {B} are pricier than {C}.'
        sent3 = f'The {D} are the cheapest.'
        ans = [A, B, C, D]
        np.random.shuffle(ans)
        while D == ans[0]:
            np.random.shuffle(ans)
        multiple_choice_targets = []
        for i in self.items:
            multiple_choice_targets.append('The ' + str(i) + ' are the most expensive.')
        
        return(f'The greengrocer sells four types of fruit: {self.items[0]}, {self.items[1]}, {self.items[2]} and {self.items[3]}. ' \
                + ' '.join([i for i in np.random.permutation([sent1, sent2, sent3])]) \
                + f'\n\nAnswer choices: \n(A) {multiple_choice_targets[0]}\n(B) {multiple_choice_targets[1]}\n(C) {multiple_choice_targets[2]}\n(D) {multiple_choice_targets[3]}\n'), B, multiple_choice_targets
    
    def _get_name_prompt(self):
        A,B,C,D = np.random.permutation(self.items)
        sent1 = f'{A} is hungrier than {B}.'
        sent2 = f'{B} is less hungry than {C}.'
        sent3 = f'{D} is the hungriest.'
        ans = [A, B, C, D]
        np.random.shuffle(ans)
        while D == ans[0]:
            np.random.shuffle(ans)

        multiple_choice_targets = []
        for i in self.items:
            multiple_choice_targets.append(str(i) + ' already ate.')
    
        return(f'Four friends are going on a picnic: {self.items[0]}, {self.items[1]}, {self.items[2]} and {self.items[3]}. ' \
                + ' '.join([i for i in np.random.permutation([sent1, sent2, sent3])]) \
                + f'\n\nAnswer choices: \n(A) {multiple_choice_targets[0]}\n(B) {multiple_choice_targets[1]}\n(C) {multiple_choice_targets[2]}\n(D) {multiple_choice_targets[3]}\n'), B, multiple_choice_targets
    
    def _get_vehicle_prompt(self):
        A,B,C,D = np.random.permutation(self.items)
        sent1 = f'The {A} is sluggish compared to the {B}.'
        sent2 = f'The {B} is quicker than the {C}.'
        sent3 = f'The {D} is the slowest.'
        ans = [A, B, C, D]
        np.random.shuffle(ans)
        while D == ans[0]:
            np.random.shuffle(ans)

        multiple_choice_targets = []
        for i in self.items:
            multiple_choice_targets.append('The ' + str(i) + ' is most likely to win.')
    
        return(f'Four vehicles are planning a race: a {self.items[0]}, a {self.items[1]}, a {self.items[2]} and a {self.items[3]}. ' \
                + ' '.join([i for i in np.random.permutation([sent1, sent2, sent3])]) \
                + f'\n\nAnswer choices: \n(A) {multiple_choice_targets[0]}\n(B) {multiple_choice_targets[1]}\n(C) {multiple_choice_targets[2]}\n(D) {multiple_choice_targets[3]}\n'), B, multiple_choice_targets

    def _get_bird_prompt(self):
        A,B,C,D = np.random.permutation(self.items)
        sent1 = f'The {A} is sings more beautifully than the {B}.'
        sent2 = f'The {B} does not sing as well as the {C}.'
        sent3 = f'The {D} has the most lovely song.'
        ans = [A, B, C, D]
        np.random.shuffle(ans)
        while D == ans[0]:
            np.random.shuffle(ans)

        multiple_choice_targets = []
        for i in self.items:
            multiple_choice_targets.append('The ' + str(i) + ' is the most painful to listen to.')
    
        return(f'Four birds are sitting on a fence: a {self.items[0]}, a {self.items[1]}, a {self.items[2]} and a {self.items[3]}. ' \
                + ' '.join([i for i in np.random.permutation([sent1, sent2, sent3])]) \
                + f'\n\nAnswer choices: \n(A) {multiple_choice_targets[0]}\n(B) {multiple_choice_targets[1]}\n(C) {multiple_choice_targets[2]}\n(D) {multiple_choice_targets[3]}\n'), B, multiple_choice_targets
    
    def _get_colour_prompt(self):
        A,B,C,D = np.random.permutation(self.items)
        sent1 = f'The {A} cat is to the left of the {B} cat.'
        sent2 = f'The {B} cat is directly on the right of the {C} cat.'
        sent3 = f'The {D} cat is on the far left.'
        ans = [A, B, C, D]
        np.random.shuffle(ans)
        while D == ans[0]:
            np.random.shuffle(ans)

        multiple_choice_targets = []
        for i in self.items:
            multiple_choice_targets.append('The ' + str(i) + ' cat is on the far right.')
            
        return(f'Four cat toys are on a table: a {self.items[0]}, a {self.items[1]}, a {self.items[2]} and a {self.items[3]}. ' \
                + ' '.join([i for i in np.random.permutation([sent1, sent2, sent3])]) \
                + f'\n\nAnswer choices: \n(A) {multiple_choice_targets[0]}\n(B) {multiple_choice_targets[1]}\n(C) {multiple_choice_targets[2]}\n(D) {multiple_choice_targets[3]}\n'), B, multiple_choice_targets

x = Question()
pprint(x.output())

{'multiple_choice_scores': [0, 0, 1, 0],
 'multiple_choice_targets': ['The green cat is on the far right.',
                             'The blue cat is on the far right.',
                             'The red cat is on the far right.',
                             'The white cat is on the far right.'],
 'parsed_inputs': 'The following paragraphs each describe a set of four items '
                  'arranged in a fixed order. The statements are logically '
                  'consistent within each paragraph. Four cat toys are on a '
                  'table: a green, a blue, a red and a white. The blue cat is '
                  'on the far left. The green cat is to the left of the red '
                  'cat. The red cat is directly on the right of the white '
                  'cat.\n'
                  '\n'
                  'Answer choices: \n'
                  '(A) The green cat is on the far right.\n'
                  '(B) The blue cat is on the far right.\n'
              

In [10]:
np.random.seed(23)

data = {'data':[]}

for i in range(100):
    x = Question()
    data['data'].append(x.output())

with open('data/bbh/logic_four/val_data.json', 'w+') as f:
    f.write(json.dumps(data))